In [0]:
!wget --output-document=imagenet-vgg-verydeep-19.mat 'https://storage.googleapis.com/marketing-files/colab-notebooks/style-transfer/imagenet-vgg-verydeep-19.mat'
# !ls

In [0]:
from google.colab import files; uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name = fn, length = len(uploaded[fn])))

In [0]:
# Import standard libraries. Set the random seed to 42 for both numpy and tensorflow to
# get the deterministic output each time you run this notebook.  
import numpy as np; np.random.seed(42); import tensorflow as tf; tf.set_random_seed(42);
import matplotlib.pyplot as plt; import pylab; import scipy.misc; from scipy import io;
import cv2; import os; from os import listdir;
%matplotlib inline

# Suppress the irrelevant warnings
import warnings; warnings.filterwarnings("ignore");

In [0]:
def load_vgg_model(path):
  
    """
    Arguments: 
    path:  Path of the mat file to load!
    
    Returns:
    graph: The complete graph with pre-trained weights and biases loaded
    """
    
#   Load the mat file and store the name of the layers somewhere!
    vgg = scipy.io.loadmat(path); vgg_layers = vgg['layers']
    
#   Get the weights corresponding to the specific layer with the corresponding name 
    def _weights(layer, expected_layer_name):
        
        """
        Arguments: 
        layer:      Integer, the layer index in the architecture
        layer_name: Name of the corresponding layer
        
        Returns:
        weight:     Pre-trained Weight 
        bias:       Pre-trained bias
        """
        
        wb = vgg_layers[0][layer][0][0][2]; 
        W = wb[0][0]; b = wb[0][1]
        layer_name = vgg_layers[0][layer][0][0][0][0]
        
        assert layer_name == expected_layer_name
        return W, b

    def _relu(conv2d_layer): 
        
        """
        Arguments: 
        conv2d_layer: Input Tensor
        
        Returns:
        output:       Output tensor with the ReLU activation added
        """
        
        output = tf.nn.relu(conv2d_layer)
        return output

    def _conv2d(prev_layer, layer, layer_name):
      
        """
        Arguments:
        prev_layer: Output activations of the previous layer
        layer:      Layer index of the architecture
        layer_name: Name of the corresponding layer
        
        Returns: 
        output:     Convoluted output
        """
        
        W, b = _weights(layer, layer_name); W = tf.constant(W); b = tf.constant(np.reshape(b, (b.size)))
        output = tf.nn.conv2d(prev_layer, filter = W, strides = [1, 1, 1, 1], padding = 'SAME') + b
        
        return output 

    def _conv2d_relu(prev_layer, layer, layer_name): 
      
        return _relu(_conv2d(prev_layer, layer, layer_name))

#   Average_pooling (Average of the elements in the sub-block) to reduce the dimensions of the activations
    def _avgpool(prev_layer): 
        
        return tf.nn.avg_pool(prev_layer, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    
#   Max_pooling (Maximum of the elements in the sub-block) to reduce the dimensions of the activations
    def _maxpool(prev_layer): 
        
        return tf.nn.max_pool(prev_layer, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

    graph = {}
    graph['input']   = tf.Variable(np.zeros((1, img_height, img_width, img_channels)), dtype = 'float32')
    graph['conv1_1']  = _conv2d_relu(graph['input'], 0, 'conv1_1')
    graph['conv1_2']  = _conv2d_relu(graph['conv1_1'], 2, 'conv1_2')
    graph['avgpool1'] = _avgpool(graph['conv1_2'])
    graph['conv2_1']  = _conv2d_relu(graph['avgpool1'], 5, 'conv2_1')
    graph['conv2_2']  = _conv2d_relu(graph['conv2_1'], 7, 'conv2_2')
    graph['avgpool2'] = _avgpool(graph['conv2_2'])
    graph['conv3_1']  = _conv2d_relu(graph['avgpool2'], 10, 'conv3_1')
    graph['conv3_2']  = _conv2d_relu(graph['conv3_1'], 12, 'conv3_2')
    graph['conv3_3']  = _conv2d_relu(graph['conv3_2'], 14, 'conv3_3')
    graph['conv3_4']  = _conv2d_relu(graph['conv3_3'], 16, 'conv3_4')
    graph['avgpool3'] = _avgpool(graph['conv3_4'])
    graph['conv4_1']  = _conv2d_relu(graph['avgpool3'], 19, 'conv4_1')
    graph['conv4_2']  = _conv2d_relu(graph['conv4_1'], 21, 'conv4_2')
    graph['conv4_3']  = _conv2d_relu(graph['conv4_2'], 23, 'conv4_3')
    graph['conv4_4']  = _conv2d_relu(graph['conv4_3'], 25, 'conv4_4')
    graph['avgpool4'] = _avgpool(graph['conv4_4'])
    graph['conv5_1']  = _conv2d_relu(graph['avgpool4'], 28, 'conv5_1')
    graph['conv5_2']  = _conv2d_relu(graph['conv5_1'], 30, 'conv5_2')
    graph['conv5_3']  = _conv2d_relu(graph['conv5_2'], 32, 'conv5_3')
    graph['conv5_4']  = _conv2d_relu(graph['conv5_3'], 34, 'conv5_4')
    graph['avgpool5'] = _avgpool(graph['conv5_4'])
    
    return graph

In [0]:
img_width = 500; img_height = 350; img_channels = 3; log_path = "./output"
model = load_vgg_model('./imagenet-vgg-verydeep-19.mat');

MEANS = np.array([123.68, 116.779, 103.939]).reshape((1, 1, 1, 3)); 
img_shape = [1, img_height, img_width, img_channels];

def reshape_and_normalize_image(image):
  
    """
    Arguments:
    image: Original image
    
    Returns:
    image: Normalized image (Image minus mean of the image datset that was used to train the model) 
           after getting reshaped to desired size
    """
    
    image = np.reshape(image, ((1,) + image.shape)); image = image - MEANS
    
    return image


content_img = cv2.imread('2 (1).jpg'); content_array = reshape_and_normalize_image(content_img);
style_img = cv2.imread('Style.jpg'); style_array = reshape_and_normalize_image(style_img);

In [0]:
def save_image(path, image):
    
    """
    Arguments:
    path:  path where output image will get saved
    image: output image after de-normalizing
    """
    
    image = image + MEANS; image = np.clip(image[0], 0, 255).astype('uint8')
    cv2.imwrite(path, image)

def generate_noise_image(content_arr, noise_ratio):
  
    """
    Arguments:
    content_arr: Original image
    noise_ratio: Noise ratio, 1 means output will be complete noise
                 0 means output will be content image
    
    Returns:     Noisier version of the original image
    """
    
    noise_img = np.random.uniform(-20., 20., size = img_shape).astype('float32')
    noise_img = noise_ratio*noise_img + (1-noise_ratio)*content_arr
    
    return noise_img

generated_img = generate_noise_image(content_array, 0)

In [0]:
STYLE_LAYERS = [('conv2_1', 0.25), ('conv3_1', 0.25), ('conv4_1', 0.25), ('conv5_1', 0.25) ]; 
CONTENT_LAYERS = [('conv3_1', 1)]

sess = tf.Session(); sess.run(tf.global_variables_initializer())

In [0]:
content_features = {}; sess.run(model['input'].assign(content_array));
for layer, coef in CONTENT_LAYERS: content_features[layer] = tf.convert_to_tensor(sess.run(model[layer]))
  
# Calculates the content loss
def content_loss(content_tensor, generated_tensor):
    
    """
    Arguments:
    content_tensor:   Activations corresponding to particular layer of content image
    generated tensor: Activations corresponding to particular layer of generated image
    
    Returns:
    content_loss:     Loss computed using the formula mentioned in ReadMe
    """
    
    m, n_h, n_w, n_c = content_tensor.get_shape().as_list()
    content_tensor_unrolled = tf.reshape(content_tensor, shape = [n_h*n_w, n_c])
    generated_tensor_unrolled = tf.reshape(generated_tensor, shape = [n_h*n_w, n_c])
    content_loss = tf.reduce_sum(tf.square(content_tensor_unrolled - generated_tensor_unrolled))/(4*n_h*n_w*n_c)
    
    return content_loss

def gram_matrix(mat):
  
    """
    Arguments:
    mat:      Original matrix
    
    Returns:
    gram_mat: Gram matrix of the original matrix computed using the formula mentioned in ReadMe
    """
    
    m, n_h, n_w, n_c = mat.get_shape().as_list(); mat_reshaped = tf.reshape(tf.transpose(mat), shape = [n_c, n_h*n_w])
    gram_mat = tf.matmul(mat_reshaped, mat_reshaped, transpose_a = False, transpose_b = True)
    
    return gram_mat

def style_loss_per_layer(s_t, g_t):
  
    """
    Arguments:
    s_t:    Activations corresponding to particular layer of style image
    g_t:    Activations corresponding to particular layer of generated image
    
    Returns:
    s_loss: Style loss computed using the formula mentioned in ReadMe
    """
    
    m, n_h, n_w, n_c = s_t.get_shape().as_list(); s_t_gram_mat = gram_matrix(s_t); g_t_gram_mat = gram_matrix(g_t)
    s_loss = tf.reduce_sum(tf.square(s_t_gram_mat - g_t_gram_mat))/(4*n_c**2*n_h**2*n_w**2)
    
    return s_loss

def Style_loss(style_features, STYLE_LAYERS):
  
    """
    Arguments:
    style_faetures: 
    STYLE_LAYERS:   Layers that have been taken into account while calculating style loss  
    
    Returns:
    style_loss:    weighted loss of different style loss per layers
    """
    style_loss = 0
    for layer, coef in STYLE_LAYERS: style_loss += coef*style_loss_per_layer(style_features[layer], model[layer])
    
    return style_loss

style_features = {}; sess.run(model['input'].assign(style_array))
for layer, coef in STYLE_LAYERS: style_features[layer] = tf.convert_to_tensor(sess.run(model[layer]))

def totalvar_loss(image): return tf.image.total_variation(image);

c_loss = content_loss(content_features['conv3_1'], model['conv3_1']); 
s_loss = Style_loss(style_features, STYLE_LAYERS)
tv_loss = totalvar_loss(tf.convert_to_tensor(generated_img, preferred_dtype = tf.float32))

In [0]:
def total_loss(content_loss, style_loss, tv_loss, alpha, beta, gamma): return alpha*content_loss + beta*style_loss + gamma*tv_loss

t_loss = total_loss(c_loss, s_loss, tv_loss, 10, 100, 10)
train_step = tf.train.AdamOptimizer(10.0).minimize(t_loss)

In [0]:
sess.run(tf.global_variables_initializer()); sess.run(model['input'].assign(generated_img))
if not os.path.exists(log_path): os.makedirs(log_path)

for i in range(1, 41):    
  sess.run(train_step);    
  if i % 10 == 0:
    Jt, Jc, Js, Jtv = sess.run([t_loss, c_loss, s_loss, tv_loss])
    print("Iteration " + str(i) + " :"); print("total cost = " + str(Jt))
    print("content cost = " + str(Jc)); print("style cost = " + str(Js))
    mixed_image = sess.run(model['input']); save_image("output/" + str(i) + ".png", mixed_image)

Iteration 10 :
total cost = [8.522509e+09]
content cost = 9349.85
style cost = 84527384.0
Iteration 20 :
total cost = [4.475226e+09]
content cost = 10605.113
style cost = 44054436.0
Iteration 30 :
total cost = [2.5190085e+09]
content cost = 10518.967
style cost = 24492270.0
Iteration 40 :
total cost = [1.6735465e+09]
content cost = 10376.962
style cost = 16037663.0


In [0]:
from google.colab import files
for i in range(1, 5): files.download('./output/' + str(10*i) + '.png')